In [16]:
import pandas as pd
import sqlite3
import csv
import os
import re
import glob

# --- 設定 ---
DB_NAME = "../anaslo_02/anaslo_02.db"
# CSV_FILE = "..anaslo_02/csv/東京都_EXA_FIRST_2025-04-03.csv"
csv_files = glob.glob("../anaslo_02/csv/*_202*.csv")
csv_file = csv_files[0]

In [17]:
# --- ファイル名から都道府県・ホール名・日付を取得 ---
filename = os.path.basename(csv_file)
match = re.match(r"(.+?)_(.+?)_(\d{4}-\d{2}-\d{2})\.csv", filename)
if not match:
    raise ValueError("ファイル名は『都道府県_ホール名_YYYY-MM-DD.csv』の形式である必要があります")

prefecture_name = match.group(1)
hall_name = match.group(2)
data_date = match.group(3)

print(f"📄 ファイル名から抽出：")
print(f"   都道府県 : {prefecture_name}")
print(f"   ホール名 : {hall_name}")
print(f"   データ日付 : {data_date}")

📄 ファイル名から抽出：
   都道府県 : 東京都
   ホール名 : EXA_FIRST
   データ日付 : 2025-04-03


In [19]:
# --- CSVを読み込む ---
df = pd.read_csv(csv_file)

# --- 整形：数値カラムのカンマや + を除去 ---
df["G数"] = df["G数"].astype(str).str.replace(",", "").astype(int)
df["差枚"] = (
    df["差枚"]
    .astype(str)
    .str.replace(",", "")
    .str.replace("+", "")
    .astype(int)
)
df.head()

,機種名,台番号,G数,差枚,BB,RB,合成確率,BB確率,RB確率
0,マイジャグラーV,1001,3137,-1912,5,4,1/348.6,1/627.4,1/784.2
1,マイジャグラーV,1002,1315,-909,3,0,1/438.3,1/438.3,1/0.0
2,マイジャグラーV,1003,2017,-126,7,5,1/168.1,1/288.1,1/403.4
3,マイジャグラーV,1004,5421,1088,24,14,1/142.7,1/225.9,1/387.2
4,マイジャグラーV,1005,681,-321,1,1,1/340.5,1/681.0,1/681.0


In [11]:
# --- データベース接続 ---
conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

# 都道府県IDの取得 or 登録
cursor.execute("INSERT OR IGNORE INTO prefectures (name) VALUES (?)", (prefecture_name,))
cursor.execute("SELECT prefecture_id FROM prefectures WHERE name = ?", (prefecture_name,))
prefecture_id = cursor.fetchone()[0]

# ホールIDの取得 or 登録
cursor.execute("""
    INSERT OR IGNORE INTO halls (name, prefecture_id)
    VALUES (?, ?)
""", (hall_name, prefecture_id))
cursor.execute("""
    SELECT hall_id FROM halls
    WHERE name = ? AND prefecture_id = ?
""", (hall_name, prefecture_id))
hall_id = cursor.fetchone()[0]

print(f"🏢 {prefecture_name}: {prefecture_id}")
print(f"🏢 {hall_name}: {hall_id}")

🏢 東京都: 13
🏢 EXA_FIRST: 2093


In [13]:
# --- CSV読み込みと登録処理 ---
with open(csv_file, newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        model_name = row["機種名"].strip()
        unit_no = int(row["台番号"])
        games = int(row["G数"].replace(",", "").strip())

        # 差枚（+/- の処理含む）
        medals_str = row["差枚"].replace(",", "").replace("+", "").strip()
        medals = int(medals_str) if medals_str else 0

        BB = int(row["BB"])
        RB = int(row["RB"])

        # 機種IDの取得 or 登録
        cursor.execute("INSERT OR IGNORE INTO models (name) VALUES (?)", (model_name,))
        cursor.execute("SELECT model_id FROM models WHERE name = ?", (model_name,))
        model_id = cursor.fetchone()[0]

        # 出玉データ登録
        cursor.execute("""
            INSERT OR IGNORE INTO results (
                hall_id, model_id, unit_no, date, start, BB, RB, medals
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (hall_id, model_id, unit_no, data_date, games, BB, RB, medals))

# --- コミットと終了 ---
conn.commit()
conn.close()

print("✅ 出玉データをデータベースに登録しました。")


KeyError: '機種名'